In [1]:
from helpers import *
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
 
file_source ='/Users/mariannecivitardevol/Documents/EPFL/Masters/Year 2/ADA copy/'
bechdel_path2 = "/Users/mariannecivitardevol/Documents/EPFL/Masters/Year 2/ADA copy/MovieSummaries/movies_feminism.csv"
MovieMetadata_df, CharacterMetadata_df, names_df, plot_summaries_df, tvTropes_df = create_datasets(file_source)

# Create a dataframe with the bechdel data as well as plots
bechdel_data2 = pd.read_csv(bechdel_path2)
# drop the NaN movie plots
plot_summaries_df = plot_summaries_df.dropna(subset=['Summaries'])
merged_bechdel_plot = bechdel_plots_dataset_creation(bechdel_data2, MovieMetadata_df,plot_summaries_df)

In [2]:
MovieMetadata_df

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages (Freebase ID:name tuples),Movie countries (Freebase ID:name tuples),Movie genres (Freebase ID:name tuples),Year
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",2000
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",1987
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",1983
...,...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",2011
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",2011
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",1972
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",1992


In [3]:
summaries = merged_bechdel_plot['Summaries'] # juste les summaries des films qui ont le bechdel test
score = merged_bechdel_plot['bt_score'] # juste les scores des films qui ont le bechdel test
new_score = []
ones = 0
zeros = 0
for sc in score:
    if sc == 3 :
        new_score.append(1)
        ones += 1
    else :
        new_score.append(0)
        zeros+=1

print('number of ones: ', ones)
print('number of zeros: ', zeros)

number of ones:  3228
number of zeros:  635


In [4]:
summaries.values[0]

"Cinderella is the much-loved only child of a widowed aristocrat. After deciding that his beloved daughter needs a mother's care, Cinderella's father marries Lady Tremaine, a proud woman with two daughters from her first marriage, Drizella and Anastasia. Plain and socially awkward, these stepsisters are bitterly envious of the beautiful and charming Cinderella. After the death of Cinderella's father, Lady Tremaine and her daughters take over the estate, and begin to abuse and mistreat Cinderella out of jealousy, and even allow their cat, Lucifer, to torment her. Despite being forced into servitude in her own home, Cinderella becomes a kind woman and befriends the animals living in the barn and many of the mice and birds who live in and around the chateau. At the royal palace, the King is distressed that his son does not intend to marry. Determined to see grandchildren, the King and the Duke organize a ball for Prince Charming in an effort to enable his son to marry, with every eligible

In [5]:
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
#shuffle the data
summaries = pd.DataFrame(summaries)
# Shuffle the DataFrame values and the associated scores
X, Y = shuffle(summaries.values, new_score, random_state=0)

# Convert X back to a DataFrame for resampling if needed
X = pd.DataFrame(X, columns=summaries.columns)

# Apply RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, Y_resampled = rus.fit_resample(X, Y)

#split into training and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## 1. Trouver les thèmes dans les déscriptions des films


In [6]:
import numpy as np
dict_train = {'label': list(Y_train), 'text': X_train['Summaries']} 
   
dataset_train = pd.DataFrame(dict_train)

dict_test = {'label': Y_test, 'text': X_test['Summaries']} 
   
dataset_test = pd.DataFrame(dict_test)
dataset_test

,label,text
1474,1,"In 1963, thirteen-year-old Anne and fifteen-ye..."
3172,1,At a ballroom of a hospital charity party in B...
2676,1,"John Kent , a former star football player at H..."
2083,1,"Lee Holloway , the socially awkward and emotio..."
1106,1,"Thousands of years ago, the 13 Primes travel t..."
...,...,...
2176,1,"Gru , a super-villain, has his pride injured w..."
416,0,The USS Tiger Shark is a submarine on patrol i...
1222,1,"Ren McCormack , a teenager raised in Chicago, ..."
217,0,"A superhuman criminal named Gabriel YuLaw , on..."


In [7]:
from datasets import Dataset
hf_dataset_train = Dataset.from_pandas(dataset_train)
hf_dataset_test = Dataset.from_pandas(dataset_test)

In [8]:
hf_dataset_train[0]

{'label': 0,
 'text': "Goodbye, Dragon Inn is set in the approximately ninety minutes of the last feature at an old Taipei cinema that is closing down, showing King Hu's 1967 sword-fighting classic Dragon Inn. Only a few people are present in the cinema, and a variety of subplots are developed around them. Throughout the film, the ticket woman tries to find the projectionist, searching for him in order to present him with a steamed bun. She wears an iron brace on her leg. She walks around the theater throughout the film, struggling up and down stairs. A young Japanese tourist wanders around the cinema in search of a homosexual encounter. An older man tells him that the cinema is haunted. An old man, who was one of the actors who appeared in the original Dragon Inn, watches the film with tears in his eyes. Outside the theater, he encounters an older man who had been watching the film with his grandson; this man also starred in the original film. The film is shot with almost no camera mo

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets_train = hf_dataset_train.map(tokenize_function, batched=True)

tokenized_datasets_test = hf_dataset_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/3090 [00:00<?, ? examples/s]

Map:   0%|          | 0/773 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [12]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
small_train_dataset = tokenized_datasets_train.shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets_test.shuffle(seed=42).select(range(100))

In [41]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train, #utiliser small_train_dataset si veut faire avec plus petit dataset
    eval_dataset=tokenized_datasets_test,#utiliser small_test_dataset si veut faire avec plus petit dataset
    compute_metrics=compute_metrics,
)
trainer.train()

  0%|          | 0/39 [00:00<?, ?it/s]

{'train_runtime': 8975.3563, 'train_samples_per_second': 0.033, 'train_steps_per_second': 0.004, 'train_loss': 0.42128934615697616, 'epoch': 3.0}


TrainOutput(global_step=39, training_loss=0.42128934615697616, metrics={'train_runtime': 8975.3563, 'train_samples_per_second': 0.033, 'train_steps_per_second': 0.004, 'total_flos': 78933316608000.0, 'train_loss': 0.42128934615697616, 'epoch': 3.0})

In [42]:
# Save the model to a specified directory
model.save_pretrained("./saved_model")

# Save the tokenizer (if you used one)
tokenizer.save_pretrained("./saved_model")


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')